In [1]:
# sudo service mysql stop
# docker run --name mysql_call -p 3306:3306 -e MYSQL_ROOT_PASSWORD=your_secret_pwd -d mysql:latest

In [17]:
import pymysql
import pandas as pd

In [2]:
with open('mysql_local.pass','r') as file:
    mysql_pass = file.read().replace('\n', '')
    file.close()

In [3]:
conn = pymysql.connect(
            host = '10.2.4.87', 
            user = 'root',
            passwd = mysql_pass,
            db = 'mysql',
            autocommit = True
        )

In [4]:
cursor = conn.cursor()

In [40]:
query = "create database 1c;"
cursor.execute(query)

1

In [5]:
query = "use 1c;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [6]:
query = "show tables;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('calls',)


In [21]:
#--create table users ( id int not null auto_increment, token varchar(36), 
#email varchar(64), event_date DATETIME, primary key (id) );
query = "create table calls"
query += " ("
query += " id int not null auto_increment,"
query += " call_date DATETIME,"
query += " ak BOOLEAN,"
query += " miko BOOLEAN,"
query += " mrm BOOLEAN,"
query += " incoming BOOLEAN,"
query += " linkedid varchar(64),"
query += " base_name varchar(64),"
query += " primary key (id)"
query += " );"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [20]:
query = "drop table calls;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [22]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='calls';"
calls = pd.read_sql(query, con = conn)
calls

,COLUMN_NAME
0,id
1,call_date
2,ak
3,miko
4,mrm
5,incoming
6,linkedid
7,base_name


In [30]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='calls';"
calls = pd.read_sql(query, con = conn)
calls

,COLUMN_NAME
0,call_date
1,ak
2,miko
3,mrm
4,incoming
5,linkedid
6,base_name


In [29]:
filename = 'data.csv'
# df = pd.read_csv(filename, ';', dtype={
#         'call_date': 'str',
#         'ak': 'bool',
#         'miko': 'bool',
#         'mrm': 'bool',
#         'incoming': 'bool',
#         'linkedid': 'str',
#         'base_name': 'str',
#     })